<a href="https://colab.research.google.com/github/sheltauro/makemore/blob/main/mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
import requests
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
read_data = requests.get('https://raw.githubusercontent.com/sheltauro/makemore/main/data/names.txt').content
read_data = read_data.splitlines()
words = [str(rd, 'UTF-8') for rd in read_data]

In [6]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [112]:
# building the dataset

def build_dataset(words):
  block_size = 3 # context length: how many characters fo we take to predict the next one?
  X, Y = [], []

  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      # print(''.join(itos[i] for i in context), '--->', ch)
      context = context[1:] + [ix] # crop and append
  X = torch.tensor(X)
  Y = torch.tensor(Y)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xval, Yval = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])
    

In [119]:
g = torch.Generator().manual_seed(2147483647)
# Embedding matrix for characters. Compress a character to a size 2 embedding.
C = torch.randn((27, 10), generator=g)
W1 = torch.randn((30, 200), generator=g)
b1 = torch.randn(200, generator=g)
W2 = torch.randn((200, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [120]:
print(sum(p.nelement() for p in parameters)) # number of parameters in total

for p in parameters:
  p.requires_grad = True

11897


In [123]:
for i in range(100000):
  # minibatch construct
  ix = torch.randint(0, Xtr.shape[0], (32,)) # picking 32 random elements

  # forward pass
  emb = C[Xtr[ix]] # (32, 3, 10)
  h = torch.tanh(emb.view(-1, 30) @ W1 + b1) # (32, 27)
  logits = h @ W2 + b2 # (32, 27)
  loss = F.cross_entropy(logits, Ytr[ix]) # softmax function.
  # print(loss.item())

  # backward pass
  for p in parameters:
    p.grad = None
  loss.backward()
  
  # update
  # lr = 0.1 if i < 100000 else 0.01
  lr = 0.01
  for p in parameters:
    p.data += -lr * p.grad

print(loss.item())

2.021294116973877


In [124]:
emb = C[Xval] # (32, 3, 10)
h = torch.tanh(emb.view(-1, 30) @ W1 + b1) # (32, 27)
logits = h @ W2 + b2 # (32, 27)
loss = F.cross_entropy(logits, Yval) # softmax function.
loss

tensor(2.1528, grad_fn=<NllLossBackward0>)